# CSCI 3343 Lab 4: Pytorch for Multilayer Perceptron Model

**Posted:** Wednesday, October 6, 2021

**Due (suggested):** Wednesday, October 13, 2021

__Total Points__: 0.5 (extra pts for the final grade)

__Name__:
[Your first name] [Your last name], [Your BC username]

(e.g. Donglai Wei, weidf)

__Submission__: please rename the .ipynb file as __\<your_username\>_lab4.ipynb__ before you submit it to canvas. Example: weidf_lab4.ipynb.

Acknowledgement: Tongzhou Wang (MIT course 6.869)

#Introduction

We will review Lecture 8 and Lecture 9 on how to build a multilayer perceptron (MLP) model and how to compute the backpropagation manually.

## Colab setup: running in GPU session
Runtime -> Change runtime type -> Hardware accelerator -> GPU

# Part 1. Build a MLP model

## 2-layer MLP (1 hidden layer)
To build a deep learning model in Pytorch, we need to define the needed layers under `__init__()` and specify the model computation under `foward()`. The gradient computation is automatically done under the parent's `backward()` (can be overwritten if needed).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP_oneHiddenLayer(nn.Module):
  def __init__(self, input_dim, output_dim, num_neuron, nonlinear=F.relu):
    super(MLP_oneHiddenLayer, self).__init__()

    self.fc1 = nn.Linear(input_dim, num_neuron)
    self.fc2 = nn.Linear(num_neuron, output_dim)
    self.nonlinear = nonlinear    
   
  def forward(self, x):
    x = torch.flatten(x, 1) # flatten all dimensions except batch
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.softmax(x, dim=1)
  
# print(MLP_oneHiddenLayer(4,3,2))

In [ ]:
! pip install hiddenlayer

In [ ]:
import hiddenlayer as hl

num_input, num_output, num_neuron = 10,20,1 
model = MLP_oneHiddenLayer(num_input, num_output, num_neuron)

hl.build_graph(model, torch.zeros([1, num_input]), transforms='')

## Exercise 1. N-layer MLP


### (a) **[TODO]** Fill in the missing code.

Let's build a MLP model with the specified number of hidden layers and number of neurons.

**Course material: Lab 4, page 11**

In [ ]:
class MLP(nn.Module):
  def __init__(self, input_dim, output_dim, num_neuron=[], nonlinear=F.relu):
    super(MLP, self).__init__()

    layers = []
    if len(num_neuron) == 0:
      layers += [nn.Linear(input_dim, output_dim)]
    else:
      layers += [nn.Linear(input_dim, num_neuron[0]), nn.ReLU()]
      for i in range(len(num_neuron)-1):
        #### TODO ####
        layers += ???
      layers += [nn.Linear(num_neuron[-1], output_dim)]    
    self.layers = nn.Sequential(*layers)
    
   
  def forward(self, x):
    x = torch.flatten(x, 1) # flatten all dimensions except batch    x = x.view(-1, 32*32*3)
    x = self.layers(x)
    return F.softmax(x, dim=1)

# test case 
num_input, num_output = 10,20
num_neuron = [128, 128, 128]
model_mlp = MLP(num_input, num_output, num_neuron)

hl.build_graph(model_mlp, torch.zeros([1, num_input]), transforms='')

# Part 2. Backpropagation by Hand (Exercise 2)

Here, let's compute the gradient for each variable by hand and compare with Pytorch's autograd result.

To get started, let's create a 2-layer MLP model (1 hidden layer) and make the forward pass.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# create a MLP model with all intermediate variables
class MLP_oneHiddenLayer_var(nn.Module):
  def __init__(self, input_dim, output_dim, num_neuron, nonlinear=F.relu):
    super(MLP_oneHiddenLayer_var, self).__init__()

    self.fc0 = nn.Linear(input_dim, num_neuron)
    self.fc1 = nn.Linear(num_neuron, output_dim)
    self.nonlinear = nonlinear    
    self.x1, self.x2, self.x3, self.x4 = None, None, None, None
   
  def forward(self, x):
    x = torch.flatten(x, 1) # flatten all dimensions except batch
    self.x1 = self.fc0(x)
    self.x2 = F.relu(self.x1)
    self.x3 = self.fc1(self.x2)
    self.x4 = F.softmax(self.x3, dim=1)

    # by default/to save memory, the gradient for non-leaf nodes
    # (intermediate variables in the computation graph) won't be saved
    self.x1.retain_grad()
    self.x2.retain_grad()
    self.x3.retain_grad()
    self.x4.retain_grad()

    return self.x4
  
model = MLP_oneHiddenLayer_var(input_dim=10, output_dim=20, num_neuron=5)
# input size: batch size x input dimension
input = torch.rand([1,10])

# forward pass
output = model(input)
target = torch.rand([1,20])
# reduction='sum': L2 norm of the difference
loss = F.mse_loss(output, target, reduction = 'sum')

# backward pass (autograd)
loss.backward()

## (a) **[TODO]** Compute the gradient of the loss layer.

**Couse material: Lab 4, page 26**

In [ ]:
grad_x4_pt = model.x4.grad

#### TODO ####
grad_x4_manual = ???

print('x4: max difference between gt and yours:', (grad_x4_pt - grad_x4_manual.reshape(-1)).abs().max())

## (b) **[TODO]** Compute the gradient of the nonlinear layer (softmax).

In class, the MLP model uses sigmoid for the binary classifaction task. For the softmax function, the input $x_3$ and output $x_4$ both have N dimension. Thus, the local derivative $\dfrac{\partial x_4}{\partial x_3}$ have the dimension NxN (remember, need to compute every pair of variables between $x_4[i]$ and $x_3[j]$), which is the Jacobian matrix.

The formula is $\dfrac{\partial x_4[i]}{\partial x_3[j]}=x_4[i](\delta_{i=j}-x_4[j])$, where $\delta_{i=j}=1$ when $i==j$ and 0 otherwise.


$\begin{align}
\dfrac{\partial x_4}{\partial x_3} &= \begin{pmatrix}x_4[0](1-x_4[0]) & -x_4[0]x_4[1] & \dots & -x_4[0]x_4[N-1]\\ -x_4[1]x_4[0] & x_4[1](1-x_4[1]) & \dots & -x_4[1]x_4[N-1]\\ \vdots & \vdots & \ddots&\vdots\\ x_4[N-1]x_4[0] & -x_4[N-1]x_4[1] & \dots & x_4[N-1](1-x_4[N-1]) \end{pmatrix}\\
&= \begin{pmatrix}x_4[0] & 0 & \dots & 0\\ 0 & x_4[1] & \dots & 0\\ \vdots & \vdots & \ddots&\vdots\\ 0 & 0 & \dots & x_4[N-1]) \end{pmatrix}- x_4^Tx_4,
\end{align}$

where $x_4$ has the dimension 1xN

[[Link for more details]](https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1)


**Couse material: Lab 4, page 27**

Hints: in Pytorch, `a @ b` means matrix multiplication (which you need!), while `a*b` means element-wise multiplication.

In [ ]:
grad_x3_pt = model.x3.grad

#### TODO ####
# jacobian matrix
dx4_dx3 = ???

grad_x3_manual = ???

print('x3: Max difference between gt and yours:', (grad_x3_pt - grad_x3_manual.reshape(-1)).abs().max())

## (c) **[TODO]** Compute the gradient of the Linear layer. 
The $W$ in the slide is the concatenation of $W$ and $b$: `W = [fc.weight, fc.bias.reshape(-1,1)]`


**Couse material: Lab 4, page 30**

In [ ]:
grad_x2_pt = model.x2.grad
grad_W1_W_pt = model.fc1.weight.grad
grad_W1_b_pt = model.fc1.bias.grad

#### TODO ####
grad_x2_manual = ???
grad_W1_W_manual = ???
grad_W1_b_manual =  ???

print('x2: max difference between gt and yours:', (grad_x2_pt - grad_x2_manual.reshape(-1)).abs().max())
print('W1_W: max difference between gt and yours:', (grad_W1_W_pt.reshape(-1) - grad_W1_W_manual.reshape(-1)).abs().max())
print('W1_b: max difference between gt and yours:', (grad_W1_b_pt.reshape(-1) - grad_W1_b_manual.reshape(-1)).abs().max())

## (d) **[TODO]** Compute the gradient of the Nonlinear layer (ReLU).

**Couse material: Lab 4, page 27&29**

In [ ]:
grad_x1_pt = model.x1.grad

#### TODO ####
grad_x1_manual = ???

print('x1: max difference between gt and yours:', (grad_x1_pt - grad_x1_manual.reshape(-1)).abs().max())

## (e) **[TODO]** Compute the gradient of the Linear layer.
This the gradient for $W_0$ and $b_0$, so that we can do gradient descent.

**Couse material: Lab 4, page 30**

In [ ]:
grad_W0_W_pt = model.fc0.weight.grad
grad_W0_b_pt = model.fc0.bias.grad

#### TODO ####
grad_W0_W_manual = ???
grad_W0_b_manual =  ???

print('W0_W: max difference between gt and yours:', (grad_W0_W_pt.reshape(-1) - grad_W0_W_manual.reshape(-1)).abs().max())
print('W0_b: max difference between gt and yours:', (grad_W0_b_pt.reshape(-1) - grad_W0_b_manual.reshape(-1)).abs().max())

# Part 3. Train a MLP model for CIFAR10 classification

The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images that are commonly used to train machine learning and computer vision algorithms. It is one of the most widely used datasets for machine learning research. The CIFAR-10 dataset contains 60,000 32x32 color images in 10 different classes. The 10 different classes represent airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. There are 6,000 images of each class. (Source: [Wikipedia](https://en.wikipedia.org/wiki/CIFAR-10))

**Course material: self-study Lab 4, page 39-45**

In [ ]:
!pip install tensorboardcolab

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets

# Important: tensorboardcolab only works with tensorflow 1.x
%tensorflow_version 1.x
from tensorboardcolab import TensorBoardColab

## 1.1 Data: Download and Dataloader

### (a) Download: use Pytorch datasets library

In [ ]:
# transform: will be applied in the iteration of the dataloader
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar10_train_val = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

cifar10_train, cifar10_val = torch.utils.data.random_split(cifar10_train_val, [45000, 5000])
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

print('Class labels:', cifar10_test.classes)

### (b) Dataloader: sample a batch
**Course material: self-study Lab 4, page 41-43**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

batch_size = 16
data_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=batch_size,
                                         shuffle=True, num_workers=1)

In [ ]:
# get some random test images
dataiter = iter(data_loader)
images, labels = dataiter.next()

# show images
imshow(make_grid(images, nrow=4))
# print labels
for i in range(4):
  print(' '.join('%5s' % cifar10_test.classes[labels[j]] for j in range(i*4, (i+1)*4)))

## 1.2 Model

### (a) Hypothesis Space (Model Architecture)

**Course material: self-study Lab 4, page 40**

In [ ]:
class MLP_basic(nn.Module):
  def __init__(self):
    super(MLP_basic, self).__init__()
    self.fc1 = nn.Linear(32*32*3, 256)
    self.fc2 = nn.Linear(256, 10)
   
  def forward(self, x):
    x = torch.flatten(x, 1) # flatten all dimensions except batch    x = x.view(-1, 32*32*3)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.softmax(x, dim=1)
    

### (b) Trainer function
For each epoch, the model goes through all the data. It sets up a tensoboard visualization to monitor the training loss in real time.

In [ ]:
class Config:  
  def __init__(self, **kwargs):
    # util
    self.batch_size = 16
    self.epochs = 0
    self.save_model_path = '' # use your google drive path to save the model
    self.log_interval = 100 # display after number of batches
    self.criterion = F.cross_entropy # loss for classification
    self.mode = 'train'
    for key, value in kwargs.items():
      setattr(self, key, value)
   
class Trainer:  
  def __init__(self, model, config, train_data = None, test_data = None):    
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.epochs = config.epochs
    self.save_model_path = config.save_model_path
    self.log_interval = config.log_interval
    self.mode = config.mode

    self.globaliter = 0    
    batch_size = config.batch_size
    if self.mode == 'train': # training mode
      self.train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=1)      
      self.tb = TensorBoardColab()
      self.optimizer = config.optimizer
    
    if test_data is not None: # need evaluation
      self.test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                         shuffle=False, num_workers=1)
    
    self.model = model.to(self.device)
    self.criterion = config.criterion # loss function
    
                
  def train(self, epoch):  
    self.model.train()
    for batch_idx, (data, target) in enumerate(self.train_loader):
      
      self.globaliter += 1
      data, target = data.to(self.device), target.to(self.device)

      self.optimizer.zero_grad()
      predictions = self.model(data)

      loss = self.criterion(predictions, target)
      loss.backward()
      self.optimizer.step()

      if batch_idx % self.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(self.train_loader.dataset),
                  100. * batch_idx / len(self.train_loader), loss.item()))
        self.tb.save_value('Train Loss', 'train_loss', self.globaliter, loss.item())
        self.tb.flush_line('train_loss')
        
        
  def test(self, epoch):
    self.model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
      print('Start testing...')
      for data, target in self.test_loader:
        data, target = data.to(self.device), target.to(self.device)
        predictions = self.model(data)
        test_loss += self.criterion(predictions, target, reduction='sum').item()
        prediction = predictions.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()

      test_loss /= len(self.test_loader.dataset)
      accuracy = 100. * correct / len(self.test_loader.dataset)

      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(self.test_loader.dataset), accuracy))
      if self.mode == 'train': # add validation data to tensorboard
        self.tb.save_value('Validation Loss', 'val_loss', self.globaliter, test_loss)
        self.tb.flush_line('val_loss')

  def main(self):    
    if self.mode == 'train':
      for epoch in range(1, self.epochs + 1):          
          self.train(epoch)
          if self.test_loader is not None:
            # exist validation data
            self.test(epoch)
    elif self.mode == 'test':
      self.test(0)
          
    if (self.save_model_path != ''):
        torch.save(self.model.state_dict(), self.save_model_path + "/cifar10_cnn.pt")

### (c) Check out the initial model accuracy. 
The performance is similar to that of the random guess (10%).

In [ ]:
# create a model
model = MLP_basic()

# Obtain its performance on the test data
test_config = Config(mode='test')
Trainer(model, test_config, test_data = cifar10_test).main()

### (d) Train the model
We will use the cross entropy loss for the multiclass prediction: `F.cross_entropy`. Let's train the model!


Hint: 
- Click on the tensorboard link and need to refresh for latest training losses
- Free feel to tweak the hyperparameters to improve the result.

**Course material: self-study Lab 4, page 40, 44-45**

In [ ]:
# model training

# set of hyperparameters
train_config = Config(    
    # dataloader
    batch_size = 16,
    # loss
    criterion = F.cross_entropy,
    # optimization
    optimizer = optim.SGD(model.parameters(), lr=0.01),    
    epochs = 10,
    # util
    save_model_path = '', # if you like, use your google drive path to save the model (mount google drive first)
    log_interval = 100 # display after number of batches
)

Trainer(model, train_config, cifar10_train, cifar10_val).main()

### (e) Test the model
After you find the best model on the validation data, test the model on the real test data.

In [ ]:
# test results
Trainer(model, test_config, test_data = cifar10_test).main()

## Exercise 3. Train your MLP models


### (a) **[TODO]** Visualize the fc layer weights for the linear classification.
For the linear classification, e.g., logistic regression, the learned weights are templates for each class. Intuitively, the model classifies the image by doing dot product to see which template the input image is close to. Let's train a model and verify this.

In [ ]:
class LogisticRegression(nn.Module):
  def __init__(self):
    super(LogisticRegression, self).__init__()
    #### TODO ####
    ??? 
   
  def forward(self, x):
    x = torch.flatten(x, 1) # flatten all dimensions except batch    x = x.view(-1, 32*32*3)    
    #### TODO ####
    ???
    return F.softmax(x, dim=1)

model_lr = LogisticRegression()
# model training

# set of hyperparameters
train_config = Config(    
    # dataloader
    batch_size = 16,
    # loss
    criterion = F.cross_entropy,
    # optimization
    optimizer = optim.SGD(model_lr.parameters(), lr=0.01),    
    epochs = 5,
    # util
    save_model_path = '', # if you like, use your google drive path to save the model (mount google drive first)
    log_interval = 100 # display after number of batches
)

Trainer(model_lr, train_config, cifar10_train, cifar10_val).main()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# visualization code
# get the weight
W =  model_lr.fc1.weight.detach().cpu().numpy()
# scale the range to 0-1
W = (W-W.min()) / (W.max()-W.min())

for i in range(10):
  plt.subplot(3, 4, i+1)
  npimg = W[i].reshape(3, 32, 32)
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.axis('off')
  plt.title(cifar10_test.classes[i])  

plt.show()

### (b) **[TODO]** Add BatchNorm
The convergence of the model is above is slow. It's hard to get a model with more than 50% accuracy. Here, let's add the magic normalization layers to speed things up.

**Course material: Lecture 10**



In [ ]:
class MLP_bn(nn.Module):
  def __init__(self):
    super(MLP_bn, self).__init__()
    self.fc1 = nn.Linear(32*32*3, 256)
    self.bn1 = torch.nn.BatchNorm1d(256)
    self.fc2 = nn.Linear(256, 10)
   
  def forward(self, x):
    x = torch.flatten(x, 1) # flatten all dimensions except batch    x = x.view(-1, 32*32*3)
    #### TODO ####
    # -> linear -> bn -> relu
    x = ???
    x = self.fc2(x)
    return F.softmax(x, dim=1)
    
model_bn = MLP_bn()
# set of hyperparameters
train_config = Config(    
    # dataloader
    batch_size = 16,
    # loss
    criterion = F.cross_entropy,
    # optimization
    optimizer = optim.SGD(model_bn.parameters(), lr=0.01),    
    epochs = 10,
    # util
    save_model_path = '', # use your google drive path to save the model
    log_interval = 100 # display after number of batches
)

Trainer(model_bn, train_config, cifar10_train, cifar10_val).main()

In [ ]:
Trainer(model_bn, test_config, test_data = cifar10_test).main()

### (c) **[TODO]** Train a deeper MLP with BatchNorm layers
With the BatchNorm layer, we can train deeper MLP models more effectively. Please define your own model and get the test result better than 50%.

Hints: Exercise 1


In [ ]:
class MLP_bn2(nn.Module):
  def __init__(self, input_dim=32*32*3, output_dim=10, num_neuron=[256]):
    super(MLP_bn2, self).__init__()
    layers = []
    #### TODO ####
    ???
    self.layers = nn.Sequential(*layers)
   
  def forward(self, x):
    x = torch.flatten(x, 1) # flatten all dimensions except batch    x = x.view(-1, 32*32*3)
    x = self.layers(x)    
    return F.softmax(x, dim=1)
    
model_bn2 = MLP_bn2()

In [ ]:
# check your defined model
print(model_bn2)

In [ ]:
# train the model with the following hyperparameters
train_config = Config(    
    # dataloader
    batch_size = 16,
    # loss
    criterion = F.cross_entropy,
    # optimization
    optimizer = optim.Adam(model_bn2.parameters(), lr=0.01),    
    epochs = 10,
    # util
    save_model_path = '', # use your google drive path to save the model
    log_interval = 100 # display after number of batches
)

Trainer(model_bn2, train_config, cifar10_train, cifar10_val).main()


In [ ]:
# evaluate the model on the test data
Trainer(model_bn2, test_config, test_data = cifar10_test).main()

In [ ]:
# hint: you can continue to train your model with a smaller learning rate if the current model seems to get stuck